In [43]:
import pandas as pd

In [44]:
anime = pd.read_csv(r'C:\Users\srira\Desktop\Ram\Data science\Datasets\Module 16 - Recommendation engine\anime.csv')
anime.head(10)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [45]:
anime.shape

(12294, 7)

In [46]:
anime.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [47]:
anime.genre

0                     Drama, Romance, School, Supernatural
1        Action, Adventure, Drama, Fantasy, Magic, Mili...
2        Action, Comedy, Historical, Parody, Samurai, S...
3                                         Sci-Fi, Thriller
4        Action, Comedy, Historical, Parody, Samurai, S...
                               ...                        
12289                                               Hentai
12290                                               Hentai
12291                                               Hentai
12292                                               Hentai
12293                                               Hentai
Name: genre, Length: 12294, dtype: object

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
#Term frequency - Inverse document frequency: is a numerical stastic that is intended to relect how important a word is to the document in a collection or corpus

In [50]:
#Creating Tfidf vectorizer to remove all the stop words and converts into vector format

In [51]:
tfidf = TfidfVectorizer(stop_words="english") #To remove stop words

In [52]:
anime['genre'].isnull().sum()

62

In [53]:
anime['genre'] = anime['genre'].fillna(" ")

In [54]:
anime['genre'].isnull().sum()

0

In [55]:
#Preparing tfidf matrix by fitting and tranforming

In [56]:
tfidf_matrix = tfidf.fit_transform(anime.genre)
tfidf_matrix

<12294x46 sparse matrix of type '<class 'numpy.float64'>'
	with 40418 stored elements in Compressed Sparse Row format>

In [57]:
tfidf_matrix.shape

(12294, 46)

In [58]:
#46 in the above result means there are 46 distince geners in total of 12294 rows

In [59]:
#With the above code we have find the similarity scores for which we have many meaures like eclidean, pearson and cosine similarity scores

In [60]:
#FOr this case we would be using cosine similarity matrix, a numeric quantity to represent similarity b/w two movies

In [61]:
#Cosine based similarity is independent of magnitude and easy to calculate

In [62]:
from sklearn.metrics.pairwise import linear_kernel

In [63]:
cosin_sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)

In [64]:
cosin_sim_matrix.shape

(12294, 12294)

In [76]:
cosin_sim_matrix[0]

array([1.        , 0.14784981, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [65]:
#This matrix basically has score computed for each and every row to each and every row of same matrix i.e. tfidf_matrix,tfidf_matrix

In [66]:
#Creating a mapping of anime name and index

In [67]:
anime_index = pd.Series(anime.index,anime.name).drop_duplicates()
anime_index

name
Kimi no Na wa.                                            0
Fullmetal Alchemist: Brotherhood                          1
Gintama°                                                  2
Steins;Gate                                               3
Gintama&#039;                                             4
                                                      ...  
Toushindai My Lover: Minami tai Mecha-Minami          12289
Under World                                           12290
Violence Gekiga David no Hoshi                        12291
Violence Gekiga Shin David no Hoshi: Inma Densetsu    12292
Yasuji no Pornorama: Yacchimae!!                      12293
Length: 12294, dtype: int64

In [68]:
anime_index['Hunter x Hunter (2011)']

6

In [81]:
anime_index.shape

(12294,)

In [86]:
def get_anime_recommendations(Name,topN):
    anime_id = anime_index[Name]
    print(enumerate(cosin_sim_matrix[anime_id]))
    cosine_scores = list(enumerate(cosin_sim_matrix[anime_id]))
    #Sorting the cosine scores based on scores
    cosine_scores = sorted(cosine_scores,key=lambda x:x[1], reverse=True)
    #Getting the top scores
    cosine_scores_top10 = cosine_scores[0:topN+1]
    
    #Getting the anime index and scores
    anime_idx = [i[0] for i in cosine_scores_top10]
    anime_scores = [i[1] for i in cosine_scores_top10]
    
    anime_similar_show = pd.DataFrame(columns=["name","Score"])
    anime_similar_show["name"] = anime.loc[anime_idx,"name"]
    anime_similar_show["Score"] = anime_scores
    anime_similar_show.reset_index(inplace=True)
    anime_similar_show.drop(["index"],axis=1,inplace=True)
    print(anime_similar_show)

In [70]:
#Enter the name and no of recommendations that you would like to get

In [87]:
get_anime_recommendations("Kimi no Na wa.",15)

                                     name     Score
0                          Kimi no Na wa.  1.000000
1             Wind: A Breath of Heart OVA  1.000000
2            Wind: A Breath of Heart (TV)  1.000000
3   Aura: Maryuuin Kouga Saigo no Tatakai  0.955317
4          Angel Beats!: Another Epilogue  0.871505
5                                Harmonie  0.871505
6           Shakugan no Shana II (Second)  0.868830
7                       Shakugan no Shana  0.868830
8                     Shakugan no Shana S  0.868830
9                               Air Movie  0.854802
10                           Touka Gettan  0.854802
11                         Mizuiro (2003)  0.854802
12                     Venus Versus Virus  0.854802
13            Rokujouma no Shinryakusha!?  0.848888
14                           Renai Boukun  0.848888
15          Kokoro ga Sakebitagatterunda.  0.838815


In [72]:
#Why the same movie is getting recommended
# because of linear_kernel(tfidf_matrix,tfidf_matrix)

In [73]:
#In gist what i did is i took 'genre' column and calculated tfidf_matrix(which basically says how important reach word is to document), then using tfidf_matrix i cumulated distance b/w one movie to other using cosine measure, this basically gives a score of one movie with all other movies. And then we select the top N movies

In [74]:
anime.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [75]:
help(linear_kernel)

Help on function linear_kernel in module sklearn.metrics.pairwise:

linear_kernel(X, Y=None, dense_output=True)
    Compute the linear kernel between X and Y.
    
    Read more in the :ref:`User Guide <linear_kernel>`.
    
    Parameters
    ----------
    X : array of shape (n_samples_1, n_features)
    
    Y : array of shape (n_samples_2, n_features)
    
    dense_output : boolean (optional), default True
        Whether to return dense output even when the input is sparse. If
        ``False``, the output is sparse if both input arrays are sparse.
    
        .. versionadded:: 0.20
    
    Returns
    -------
    Gram matrix : array of shape (n_samples_1, n_samples_2)

